In [1]:
!pip install pyspark

In [45]:
import pandas as pd

# Load the main sales data
sales_df = pd.read_excel("nike_data.xlsx", sheet_name='Nike_Ventas')

# Load mapping sheets (you may need to adjust sheet names)
dept_map = pd.read_excel('nike_data.xlsx', sheet_name='Departamentos')
cat_map = pd.read_excel('nike_data.xlsx', sheet_name='Categorías')
region_map = pd.read_excel('nike_data.xlsx', sheet_name='Región')
prod_type_map = pd.read_excel('nike_data.xlsx', sheet_name='Tipo_Producto')
brand_map = pd.read_excel('nike_data.xlsx', sheet_name='Marca')

In [57]:
missing_size_rows = sales_df[sales_df['PRODUCT_SIZE'].isna()]

print(missing_size_rows.head(5)[['INVOICE DATE', 'id_Department', 'id_DESCRIPTION', 'id_Category', 'id_PRODUCT_TYPE','id_PRODUCT_NAME','id_SubCategory','PRODUCT_SIZE']])

       INVOICE DATE  id_Department  id_DESCRIPTION  id_Category  \
5964     2024-11-12              2             745            3   
7796     2024-10-29              3             154            3   
41883    2024-02-01              2             745            3   
56997    2023-10-05              2             154            3   
192106   2020-10-27              2             154            3   

        id_PRODUCT_TYPE  id_PRODUCT_NAME  id_SubCategory PRODUCT_SIZE  
5964                  4              839              16          NaN  
7796                  4             1638               3          NaN  
41883                 4              839               3          NaN  
56997                 4             1638               6          NaN  
192106                4             1638               3          NaN  


In [58]:
# Drop rows with missing PRODUCT_SIZE
sales_df = sales_df.dropna(subset=['PRODUCT_SIZE'])

In [59]:
sales_df.isnull().sum()

,0
INVOICE DATE,0
id_Region,0
id_State,0
id_City,0
id_Department,0
id_Category,0
id_SubCategory,0
id_PRODUCT_NAME,0
id_DESCRIPTION,0
id_PRODUCT_TYPE,0


In [46]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder.appName('NikeSalesAnalysis').getOrCreate()

# Convert pandas DataFrame to Spark DataFrame
sales_sdf = spark.createDataFrame(sales_df)

In [49]:
from pyspark.sql.functions import col

# Create Spark DataFrames
dept_sdf     = spark.createDataFrame(dept_map)
cat_sdf      = spark.createDataFrame(cat_map)
region_sdf   = spark.createDataFrame(region_map)
prodtype_sdf = spark.createDataFrame(prod_type_map)
brand_sdf    = spark.createDataFrame(brand_map)

# 1) Merge Department (using aliases)
sales_sdf = (
    sales_sdf.alias("s")
    .join(
        dept_sdf.alias("d"),
        col("s.id_Department") == col("d.id_Department"),   # same name in both
        "left"
    )
    .drop(col("d.id_Department"))        # drop from dimension
    .withColumnRenamed("DEPARTMENT", "Department")
)

# 2) Merge Category
sales_sdf = (
    sales_sdf.alias("s")
    .join(
        cat_sdf.alias("c"),
        col("s.id_Category") == col("c.id_Category"),
        "left"
    )
    .drop(col("c.id_Category"))
    .withColumnRenamed("CATEGORY", "Category")
)

# 3) Merge Region
sales_sdf = (
    sales_sdf.alias("s")
    .join(
        region_sdf.alias("r"),
        col("s.id_Region") == col("r.id_Region"),
        "left"
    )
    .drop(col("r.id_Region"))
    .withColumnRenamed("REGION", "Region_Name")
)

# 4) Merge Product Type
sales_sdf = (
    sales_sdf.alias("s")
    .join(
        prodtype_sdf.alias("p"),
        col("s.id_PRODUCT_TYPE") == col("p.id_PRODUCT_TYPE"),
        "left"
    )
    .drop(col("p.id_PRODUCT_TYPE"))
    .withColumnRenamed("PRODUCT_TYPE", "Product_Type")
)

# 5) Merge Brand
sales_sdf = (
    sales_sdf.alias("s")
    .join(
        brand_sdf.alias("b"),
        col("s.id_BRAND") == col("b.id_BRAND"),
        "left"
    )
    .drop(col("b.id_BRAND"))
    .withColumnRenamed("BRAND", "Brand")
)

sales_sdf.printSchema()


root
 |-- INVOICE DATE: timestamp (nullable = true)
 |-- id_Region: long (nullable = true)
 |-- id_State: long (nullable = true)
 |-- id_City: long (nullable = true)
 |-- id_Department: long (nullable = true)
 |-- id_Category: long (nullable = true)
 |-- id_SubCategory: long (nullable = true)
 |-- id_PRODUCT_NAME: long (nullable = true)
 |-- id_DESCRIPTION: long (nullable = true)
 |-- id_PRODUCT_TYPE: long (nullable = true)
 |-- PRODUCT_SIZE: string (nullable = true)
 |-- id_Color: long (nullable = true)
 |-- id_BRAND: long (nullable = true)
 |-- id_AVAILABILITY: long (nullable = true)
 |-- Price per Unit: double (nullable = true)
 |-- Units Sold: long (nullable = true)
 |-- Costo Unitario: double (nullable = true)
 |-- Operating Margin: double (nullable = true)
 |-- Department: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Region_Name: string (nullable = true)
 |-- Product_Type: string (nullable = true)
 |-- Brand: string (nullable = true)



In [50]:
# Few rows
sales_sdf.show(5, truncate=False)

+-------------------+---------+--------+-------+-------------+-----------+--------------+---------------+--------------+---------------+------------+--------+--------+---------------+--------------+----------+--------------+----------------+----------+--------+-----------+------------+------+
|INVOICE DATE       |id_Region|id_State|id_City|id_Department|id_Category|id_SubCategory|id_PRODUCT_NAME|id_DESCRIPTION|id_PRODUCT_TYPE|PRODUCT_SIZE|id_Color|id_BRAND|id_AVAILABILITY|Price per Unit|Units Sold|Costo Unitario|Operating Margin|Department|Category|Region_Name|Product_Type|Brand |
+-------------------+---------+--------+-------+-------------+-----------+--------------+---------------+--------------+---------------+------------+--------+--------+---------------+--------------+----------+--------------+----------------+----------+--------+-----------+------------+------+
|2024-12-30 00:00:00|1        |42      |32     |3            |2          |8             |1781           |1685         

In [51]:
# Describe numeric columns
sales_sdf.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------+------------------+------------------+-------------------+-----------------+------------------+-----------------+-------------------+----------+--------------------+-----------+------------+---------+
|summary|         id_Region|          id_State|           id_City|     id_Department|       id_Category|   id_SubCategory|   id_PRODUCT_NAME|   id_DESCRIPTION|   id_PRODUCT_TYPE|PRODUCT_SIZE|          id_Color|          id_BRAND|    id_AVAILABILITY|   Price per Unit|        Units Sold|   Costo Unitario|   Operating Margin|Department|            Category|Region_Name|Product_Type|    Brand|
+-------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------+------------------+------

In [52]:
# Count rows
print("Row count:", sales_sdf.count())

Row count: 229471


In [53]:
print(dept_map,"\n")
print(cat_map,"\n")
print(region_map,"\n")
print(prod_type_map,"\n")
print(brand_map)

  DEPARTMENT  id_DEPARTMENT
0       Kids              1
1        Men              2
2      Women              3 

                    CATEGORY  id_Category
0  Accessories and Equipment            1
1                   Clothing            2
2                      Shoes            3 

   id_Region     Region
0          1  Northeast
1          2      South
2          3       West
3          4    Midwest
4          5  Southeast 

  PRODUCT_TYPE  id_PRODUCT_TYPE
0  ACCESSORIES                1
1      APPAREL                2
2    EQUIPMENT                3
3     FOOTWEAR                4 

             BRAND  id_BRAND
0              ACG         1
1           Jordan         2
2             Nike         3
3         Nike Pro         4
4  Nike Sportswear         5
5           NIKEiD         6
6          NikeLab         7
7        No record         8


In [54]:
# Inspect all columns
print(sales_sdf.columns)

# Drop duplicated ID columns coming from mapping tables if they exist
cols_to_drop = [
    "id_DEPARTMENT",  # from dept map
    "idCategory",     # from cat map, if duplicated
    "idRegion",       # from region map, if duplicated copy
    "idPRODUCTTYPE",  # from product type map copy
    "idBRAND"         # from brand map copy
]

# Keep only those that are present
cols_to_drop = [c for c in cols_to_drop if c in sales_sdf.columns]

sales_sdf = sales_sdf.drop(*cols_to_drop)


['INVOICE DATE', 'id_Region', 'id_State', 'id_City', 'id_Department', 'id_Category', 'id_SubCategory', 'id_PRODUCT_NAME', 'id_DESCRIPTION', 'id_PRODUCT_TYPE', 'PRODUCT_SIZE', 'id_Color', 'id_BRAND', 'id_AVAILABILITY', 'Price per Unit', 'Units Sold', 'Costo Unitario', 'Operating Margin', 'Department', 'Category', 'Region_Name', 'Product_Type', 'Brand']


In [55]:
from pyspark.sql.functions import col, sum

# Count nulls per column
null_counts = sales_sdf.select([
    col(c).isNull().cast("int").alias(c) for c in sales_sdf.columns
])

null_counts_agg = null_counts.agg(
    *[sum(col(c)).alias(c) for c in null_counts.columns]
)
null_counts_agg.show()

+------------+---------+--------+-------+-------------+-----------+--------------+---------------+--------------+---------------+------------+--------+--------+---------------+--------------+----------+--------------+----------------+----------+--------+-----------+------------+-----+
|INVOICE DATE|id_Region|id_State|id_City|id_Department|id_Category|id_SubCategory|id_PRODUCT_NAME|id_DESCRIPTION|id_PRODUCT_TYPE|PRODUCT_SIZE|id_Color|id_BRAND|id_AVAILABILITY|Price per Unit|Units Sold|Costo Unitario|Operating Margin|Department|Category|Region_Name|Product_Type|Brand|
+------------+---------+--------+-------+-------------+-----------+--------------+---------------+--------------+---------------+------------+--------+--------+---------------+--------------+----------+--------------+----------------+----------+--------+-----------+------------+-----+
|           0|        0|       0|      0|            0|          0|             0|              0|             0|              0|           0|

# Analysis

# Unit Sold by Region

In [60]:
from pyspark.sql.functions import sum, col

# Sales by Region
sales_by_region = sales_sdf.groupBy('Region_Name').agg(sum('Units Sold').alias('Total Units Sold')).orderBy(col('Total Units Sold').desc())
sales_by_region.show()

+-----------+----------------+
|Region_Name|Total Units Sold|
+-----------+----------------+
|       West|        14706774|
|  Northeast|        14350079|
|    Midwest|        11330008|
|      South|        10390202|
|  Southeast|         7380896|
+-----------+----------------+



## Sales by Region (Units Sold)

The regional breakdown shows a clear hierarchy in Nike’s demand across the United States.

- The West is the strongest market, with about 14.7 million units sold, indicating very high customer penetration and store/online activity in this region.
- The Northeast is a close second at roughly 14.3 million units, showing that demand on the East Coast is almost as strong as in the West.
- The Midwest and South form a solid middle tier, with around 11.3 million and 10.4 million units sold respectively, suggesting stable but slightly lower demand compared to the coastal regions.
- The Southeast has the lowest volume at about 7.4 million units, highlighting it as the key growth opportunity region for future marketing and store expansion.

Overall, Nike’s unit sales are concentrated in the West and Northeast, while the Southeast stands out as an underpenetrated market with potential for targeted campaigns and distribution improvements.


# Total Revenue by Region

In [61]:
revenue_by_region = sales_sdf.groupBy('Region_Name').agg(
    sum(col('Price per Unit') * col('Units Sold')).alias('Total_Revenue')
).orderBy(col('Total_Revenue').desc())
revenue_by_region.show()

+-----------+--------------------+
|Region_Name|       Total_Revenue|
+-----------+--------------------+
|       West|1.1709649776500523E9|
|  Northeast|1.1365649080000443E9|
|    Midwest| 9.051104934500234E8|
|      South| 8.232798847500119E8|
|  Southeast| 5.887356644499958E8|
+-----------+--------------------+



## Total Revenue by Region

This analysis shows how Nike’s sales revenue is distributed across regions in the United States.

- The West generates the highest revenue, at roughly \$1.17 billion, confirming it as Nike’s most valuable and mature region in this dataset.  
- The Northeast follows closely with around \$1.14 billion in revenue, indicating a very strong second core market with demand close to the West.  
- The Midwest and South contribute solid revenue of about \$905 million and \$823 million respectively, forming a strong middle tier of performance.  
- The Southeast has the lowest revenue at approximately \$589 million, highlighting it as the main growth opportunity where targeted marketing and channel expansion could lift future sales.

Overall, revenue is heavily concentrated in the West and Northeast, while the Southeast appears underdeveloped relative to other regions and could be prioritized for strategic investment.


# Monthly Sales Trend Analysis

In [62]:
from pyspark.sql.functions import month, year

sales_sdf = sales_sdf.withColumn('Year', year('INVOICE DATE')).withColumn('Month', month('INVOICE DATE'))
monthly_sales = sales_sdf.groupBy('Year', 'Month').agg(
    sum('Units Sold').alias('Units_Sold')
).orderBy('Year', 'Month')
monthly_sales.show()

+----+-----+----------+
|Year|Month|Units_Sold|
+----+-----+----------+
|2020|    1|    879411|
|2020|    2|    983751|
|2020|    3|   1026025|
|2020|    4|    998164|
|2020|    5|    996424|
|2020|    6|    954075|
|2020|    7|    956987|
|2020|    8|   1002070|
|2020|    9|    953297|
|2020|   10|   1109210|
|2020|   11|    943226|
|2020|   12|    974773|
|2021|    1|   1023564|
|2021|    2|    823404|
|2021|    3|    968933|
|2021|    4|    935109|
|2021|    5|    970748|
|2021|    6|   1008172|
|2021|    7|   1056989|
|2021|    8|    927858|
+----+-----+----------+
only showing top 20 rows



## Monthly Sales Trend (Units Sold)

The monthly view shows that Nike’s demand stays consistently high across the year, with clear peaks in certain months.

- Across 2020 and 2021, most months are in the 900k–1.1M units range, indicating a stable baseline level of demand without extreme volatility.  
- There are noticeable spikes in October and early in the year (e.g., January and March), which could be linked to seasonal events like new product launches, back‑to‑school, or holiday-related promotions.  
- Some softer months, such as February and a few summer months, still maintain strong volumes, but slightly below the peak months, hinting at mild seasonality rather than sharp off‑season drops.

Overall, the trend suggests that Nike’s sales are resilient throughout the year, with certain months offering opportunities for targeted campaigns to amplify already strong demand.


# Best-Selling Product Types

In [63]:
best_selling_type = sales_sdf.groupBy('Product_Type').agg(
    sum('Units Sold').alias('Total_Units_Sold')
).orderBy(col('Total_Units_Sold').desc())
best_selling_type.show()


+------------+----------------+
|Product_Type|Total_Units_Sold|
+------------+----------------+
|     APPAREL|        30850650|
|    FOOTWEAR|        25487915|
|   EQUIPMENT|         1819069|
| ACCESSORIES|             325|
+------------+----------------+



## Best-Selling Product Types

This summary shows which product types drive most of Nike’s unit sales.

- Apparel is the top-performing category with about 30.9 million units sold, making it the primary volume driver in the portfolio.  
- Footwear is a strong second at roughly 25.5 million units, confirming that shoes are also a major contributor to overall demand.  
- Equipment sells around 1.8 million units, playing a smaller but still meaningful supporting role.  
- Accessories have very low recorded volume (325 units), which may indicate either a narrow assortment in this dataset or that accessories are not a major focus in these transactions.

Overall, most units come from Apparel and Footwear, so these categories should be the main focus for inventory planning, marketing, and forecasting.


# Top Brands by Revenue

In [64]:
top_brands = sales_sdf.groupBy('Brand').agg(
    sum(col('Price per Unit') * col('Units Sold')).alias('Revenue')
).orderBy(col('Revenue').desc())
top_brands.show()

+---------------+--------------------+
|          Brand|             Revenue|
+---------------+--------------------+
|           Nike| 1.970998567299748E9|
|Nike Sportswear|1.5500440206499407E9|
|         Jordan| 6.088943794500247E8|
|         NIKEiD|2.6706430459999746E8|
|        NikeLab|1.7921770739999938E8|
|       Nike Pro| 4.401227270000022E7|
|      No record|  4023169.8500000015|
|            ACG|   401506.3499999999|
+---------------+--------------------+



## Top Brands by Revenue

This result shows which brands generate the most sales revenue in the dataset.

- Nike [finance:Nike, Inc.] is the clear leader, with around \$1.97 billion in revenue, making it the dominant brand in the portfolio.  
- Nike Sportswear contributes roughly \$1.55 billion, showing that lifestyle and sportswear lines are also major revenue engines.  
- Jordan adds about \$609 million, confirming it as a strong sub-brand with significant commercial impact.  
- NIKEiD, NikeLab and Nike Pro together add meaningful additional revenue, while “No record” and ACG contribute only a small fraction, likely representing fringe or poorly coded entries.

Overall, most revenue is concentrated in the core Nike, Nike Sportswear and Jordan brands, which should be the main focus for strategic planning and forecasting.


# Department Performance Analysis

In [65]:
department_performance = sales_sdf.groupBy('Department').agg(
    sum('Units Sold').alias('Units_Sold'),
    sum('Operating Margin').alias('Total_Operating_Margin')
).orderBy(col('Units_Sold').desc())
department_performance.show()

+----------+----------+----------------------+
|Department|Units_Sold|Total_Operating_Margin|
+----------+----------+----------------------+
|       Men|  30151258|    62175.309999998266|
|     Women|  16724619|     34583.40999999474|
|      Kids|  11282082|     23688.23999999817|
+----------+----------+----------------------+



## Department Performance (Units and Margin)

This analysis compares how each department (Men, Women, Kids) performs in terms of volume and profitability.

- The Men’s department leads with about 30.15 million units sold and the highest total operating margin, making it the most important segment for both sales volume and profit contribution.  
- The Women’s department sells roughly 16.72 million units with a strong total operating margin, indicating a solid and profitable second pillar of the business.  
- The Kids department, with about 11.28 million units and the lowest total operating margin, is the smallest but still meaningful segment, and could benefit from targeted growth and margin optimisation strategies.

Overall, Men’s products are the primary driver, Women’s offer substantial additional value, and Kids represents a promising area for expansion and marketing focus.


# Customer Segmentation by Buying Behavior

In [66]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

features = ['Units Sold', 'Price per Unit', 'Operating Margin']
assembler = VectorAssembler(inputCols=features, outputCol="features")
segmented_data = assembler.transform(sales_sdf)
kmeans = KMeans().setK(5).setSeed(1)
model = kmeans.fit(segmented_data)
clusters = model.transform(segmented_data)
clusters.groupBy('prediction').count().show()


+----------+-----+
|prediction|count|
+----------+-----+
|         1|13287|
|         3|70542|
|         4|28677|
|         2|84946|
|         0|32019|
+----------+-----+



## Customer Segmentation by Buying Pattern (KMeans)

Using KMeans clustering on Units Sold, Price per Unit and Operating Margin, the dataset was grouped into five distinct customer/product behavior segments.

- The largest clusters are segments 2 and 3 (around 84k and 70k records), representing the most common combinations of price, volume and margin, likely corresponding to core mid-range products with steady demand.  
- Cluster 0 and cluster 4 (about 32k and 29k records) form medium-sized groups, which may capture either higher-priced or lower-volume niches, depending on their average feature values.  
- Cluster 1 is the smallest segment with around 13k records, suggesting a more specialised pattern (for example, very high or very low units sold, premium price points, or distinct margin behaviour) that could warrant targeted analysis.

Overall, this segmentation reveals that most transactions fall into a few dominant patterns, while a smaller niche cluster may indicate special product lines or customer groups worth separate marketing or pricing strategies.


# Margin Optimization Analysis

In [67]:
from pyspark.sql.functions import avg, sum, desc

margin_opt = sales_sdf.groupBy('Product_Type').agg(
    avg('Operating Margin').alias('Avg_Margin'),
    avg('Price per Unit').alias('Avg_Price'),
    sum('Units Sold').alias('Total_Sold')
).orderBy(desc('Avg_Margin'))
margin_opt.show()


+------------+------------------+------------------+----------+
|Product_Type|        Avg_Margin|         Avg_Price|Total_Sold|
+------------+------------------+------------------+----------+
| ACCESSORIES|              0.57|             13.95|       325|
|    FOOTWEAR|0.5251757979516554|108.90993636283643|  25487915|
|   EQUIPMENT|0.5248230088495592|19.935413487945087|   1819069|
|     APPAREL|0.5246576921190338| 59.01024961998738|  30850650|
+------------+------------------+------------------+----------+



## Margin Optimisation by Product Type

This table compares average margin, average price and total units sold for each product type.

- Accessories have the highest average margin (0.57) but extremely low volume (325 units), suggesting they are high-margin add-ons rather than core volume drivers.  
- Footwear combines a strong average margin (0.525) with a high average price (\$108.91) and large volume (25.49M units), making it one of the most profitable and strategically important categories.  
- Apparel and Equipment have similar average margins (0.525), but Apparel sells far more units (30.85M vs. 1.82M), so small improvements in Apparel margin can have a big impact on total profit.  

Overall, Accessories show premium margin on tiny volume, while Footwear and Apparel balance good margins with very high demand, making them prime candidates for margin optimisation and pricing strategy.


# Correlation Analysis between Price, Units Sold, and Margin

In [68]:
sales_pd = sales_sdf.select('Price per Unit', 'Units Sold', 'Operating Margin').toPandas()
correlation_matrix = sales_pd.corr()
print(correlation_matrix)


                  Price per Unit  Units Sold  Operating Margin
Price per Unit          1.000000   -0.005403          0.000426
Units Sold             -0.005403    1.000000         -0.000067
Operating Margin        0.000426   -0.000067          1.000000


## Correlation Between Price, Units Sold, and Margin

The correlation matrix shows almost no linear relationship between price, units sold, and operating margin.

- The correlation between Price per Unit and Units Sold is around -0.005, essentially zero, meaning higher prices do not systematically reduce units sold in this dataset.  
- Price per Unit and Operating Margin have a correlation of about 0.0004, and Units Sold and Operating Margin about -0.00007, both effectively zero, indicating margin is set largely independently of price and volume.  

Overall, these near-zero correlations suggest that pricing, demand (units), and margin are controlled by business rules or product strategy rather than following simple linear relationships in the raw data.


# Region–Category Performance Matrix

In [69]:
from pyspark.sql.functions import sum, col

# Revenue and units by Region and Product Category
region_category_perf = (
    sales_sdf
    .groupBy("Region_Name", "Category")
    .agg(
        sum(col("Units Sold")).alias("Total_Units_Sold"),
        sum(col("Price per Unit") * col("Units Sold")).alias("Total_Revenue")
    )
    .orderBy("Region_Name", col("Total_Revenue").desc())
)

region_category_perf.show(50, truncate=False)


+-----------+-------------------------+----------------+--------------------+
|Region_Name|Category                 |Total_Units_Sold|Total_Revenue       |
+-----------+-------------------------+----------------+--------------------+
|Midwest    |Shoes                    |4992125         |5.452356622499943E8 |
|Midwest    |Clothing                 |6206137         |3.559392559999937E8 |
|Midwest    |Accessories and Equipment|131746          |3935575.200000001   |
|Northeast  |Shoes                    |6304887         |6.856759652500055E8 |
|Northeast  |Clothing                 |7897000         |4.4655816609999156E8|
|Northeast  |Accessories and Equipment|148192          |4330776.65          |
|South      |Shoes                    |4450075         |4.8595157829999506E8|
|South      |Clothing                 |5826931         |3.340778684999958E8 |
|South      |Accessories and Equipment|113196          |3250437.950000001   |
|Southeast  |Shoes                    |3206144         |3.487136

## Region–Category Performance (Units and Revenue)

This view shows how each product category performs within every region, in terms of both units and revenue.

- In every region, Shoes and Clothing dominate, with Shoes typically generating the highest revenue and Clothing providing very strong supporting volume and sales. Accessories and Equipment contribute only a small fraction of units and revenue everywhere, confirming they are a niche category in this dataset.  
- The West and Northeast stand out: their Shoes and Clothing lines reach the highest revenues (e.g., West Shoes ~\$709M, Northeast Shoes ~\$686M), reinforcing these regions as Nike’s most valuable footwear and apparel markets.  
- The Southeast has lower Shoes and Clothing revenue compared to other regions, again highlighting it as an underpenetrated area where strengthening the core categories could unlock growth.

Overall, core performance comes from Shoes and Clothing across all regions, with regional strength peaking in the West and Northeast and clear upside potential in the Southeast.


# Price Elasticity Proxy (Price Bands vs Units Sold)

In [70]:
from pyspark.sql.functions import when, sum, col

# Create price bands
salessdf_band = (
    sales_sdf
    .withColumn(
        "Price_Band",
        when(col("Price per Unit") < 50, "<50")
        .when((col("Price per Unit") >= 50) & (col("Price per Unit") < 100), "50-99")
        .when((col("Price per Unit") >= 100) & (col("Price per Unit") < 150), "100-149")
        .otherwise("150+")
    )
)

# Aggregate units sold by price band and product type
price_band_perf = (
    salessdf_band
    .groupBy("Product_Type", "Price_Band")   # note Product_Type
    .agg(
        sum("Units Sold").alias("Total_Units_Sold"),
        sum(col("Price per Unit") * col("Units Sold")).alias("Total_Revenue")
    )
    .orderBy("Product_Type", "Price_Band")
)

price_band_perf.show(50, truncate=False)


+------------+----------+----------------+--------------------+
|Product_Type|Price_Band|Total_Units_Sold|Total_Revenue       |
+------------+----------+----------------+--------------------+
|ACCESSORIES |<50       |325             |4533.75             |
|APPAREL     |100-149   |1347925         |1.5851756274999893E8|
|APPAREL     |150+      |620332          |1.7888328054999945E8|
|APPAREL     |50-99     |13358933        |9.184280213500594E8 |
|APPAREL     |<50       |15523460        |5.574722460000423E8 |
|EQUIPMENT   |100-149   |15432           |2046218.4           |
|EQUIPMENT   |150+      |20487           |4504250.65          |
|EQUIPMENT   |50-99     |32233           |2263830.35          |
|EQUIPMENT   |<50       |1750917         |2.7447414150000144E7|
|FOOTWEAR    |100-149   |10174784        |1.249173199799998E9 |
|FOOTWEAR    |150+      |4519497         |8.506942752500176E8 |
|FOOTWEAR    |50-99     |7371539         |5.399958360500244E8 |
|FOOTWEAR    |<50       |3422095        

## Price Bands vs Product Types (Demand and Revenue)

This analysis shows how demand and revenue change across price bands for each product type.

- Apparel and Footwear sell strongly in both low and mid price bands: for Apparel, the bulk of units are in `<50` and `50-99`, but high price bands (`100-149`, `150+`) still generate very large revenues, showing customers accept premium pricing for certain lines.  
- Footwear has particularly strong revenue in the `100-149` and `150+` bands (over \$1.2B and \$850M respectively), meaning premium shoes are a major profit engine despite their higher prices.  
- Equipment sells mostly in `<50`, with smaller but still meaningful volumes in higher bands, while Accessories appear only in the `<50` band with very small volume and revenue.

Overall, lower bands drive volume, while mid and high price bands—especially for Footwear and Apparel—contribute disproportionately to revenue, indicating clear room for premium positioning and upselling strategies.


# Time-Based Forecast Feature (Next-Month Rolling Projection)

In [71]:
from pyspark.sql.functions import year, month, avg, sum
from pyspark.sql.window import Window

# Monthly units per Region
monthly_region = (
    sales_sdf
    .withColumn("Year", year("INVOICE DATE"))
    .withColumn("Month", month("INVOICE DATE"))
    .groupBy("Region_Name", "Year", "Month")   # use Region_Name
    .agg(sum("Units Sold").alias("Units_Sold"))
)

# 3-month rolling average as a forecast proxy
window_spec = (
    Window
    .partitionBy("Region_Name")                # use Region_Name
    .orderBy("Year", "Month")
    .rowsBetween(-2, 0)
)

monthly_region_forecast = (
    monthly_region
    .withColumn("Rolling_3M_Avg_Units", avg("Units_Sold").over(window_spec))
    .orderBy("Region_Name", "Year", "Month")
)

monthly_region_forecast.show(50, truncate=False)


+-----------+----+-----+----------+--------------------+
|Region_Name|Year|Month|Units_Sold|Rolling_3M_Avg_Units|
+-----------+----+-----+----------+--------------------+
|Midwest    |2020|1    |171343    |171343.0            |
|Midwest    |2020|2    |243834    |207588.5            |
|Midwest    |2020|3    |140754    |185310.33333333334  |
|Midwest    |2020|4    |213085    |199224.33333333334  |
|Midwest    |2020|5    |205554    |186464.33333333334  |
|Midwest    |2020|6    |133782    |184140.33333333334  |
|Midwest    |2020|7    |242336    |193890.66666666666  |
|Midwest    |2020|8    |170497    |182205.0            |
|Midwest    |2020|9    |212993    |208608.66666666666  |
|Midwest    |2020|10   |246968    |210152.66666666666  |
|Midwest    |2020|11   |160174    |206711.66666666666  |
|Midwest    |2020|12   |185692    |197611.33333333334  |
|Midwest    |2021|1    |170192    |172019.33333333334  |
|Midwest    |2021|2    |189795    |181893.0            |
|Midwest    |2021|3    |149043 

## Rolling 3-Month Average Units by Region

This rolling 3‑month average gives a smoothed view of demand over time for each region, helping to approximate short‑term forecasts.

- For the Midwest, monthly units sold typically fluctuate around ~170k–210k units, but the 3‑month rolling average stays in a narrower band (roughly 175k–200k), smoothing out short spikes and dips.  
- Periods where the rolling average trends upward (for example after strong months like July or December) suggest sustained growth momentum, while downward moves after weaker months indicate cooling demand.  

Overall, the rolling 3‑month average provides a more stable signal than raw monthly units, making it useful as a simple proxy for near‑term demand forecasting and inventory planning at regional level.
